In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import pandas as pd
from common.db_config import db_config
from tqdm.notebook import tqdm
from common.data_processor import *
from common.utils import split_data
import optuna.integration.lightgbm as lgb_o
import lightgbm as lgb

In [13]:
r = Results.read_db(db_config['main'], begin_date=20150101, end_date=20211231, flat_only=True)
r.data_p.head()

,horse_no,frame_no,arriving_order,horse_id,impost,jockey_id,trainer_id,date,place_id,hold_no,...,turn,ground,weather,horse_num,month,sex,age,weight,weight_change,win_prise
race_id,,,,,,,,,,,,,,,,,,,,,
201601010101,6,6,1,2014102322,52.0,01151,01024,2016-07-30,1,1,...,右,良,曇,10,7,牝,2,434,-4,500.0
201601010101,2,2,2,2014105971,54.0,00700,01066,2016-07-30,1,1,...,右,良,曇,10,7,牡,2,438,-2,500.0
201601010101,5,5,3,2014100723,54.0,01091,00359,2016-07-30,1,1,...,右,良,曇,10,7,牡,2,458,-2,500.0
201601010101,4,4,4,2014105192,51.0,01163,01046,2016-07-30,1,1,...,右,良,曇,10,7,牡,2,500,-4,500.0
201601010101,3,3,5,2014106056,54.0,01014,01058,2016-07-30,1,1,...,右,良,曇,10,7,牡,2,468,2,500.0


In [7]:
hr = HorseResults.read_db(db_config['main'])
hr.data_p.head()

,race_id,date,place_id,weather,race_no,horse_no,win_odds,popularity,arriving_order,ground,...,race_type,distance,ground,time_diff,corner_pass,last_three_furlong,prise,horse_num,first_corner,last_corner
horse_id,,,,,,,,,,,,,,,,,,,,,
2005105204,201605050508,2016-11-19,5,小雨,8,1,26.1,0.428571,0.857143,稍,...,障害,3110,稍,4.4,3-3-5-6,14.1,0.0,14.0,0.214286,0.428571
2005105204,201604030104,2016-10-15,4,晴,4,13,3.0,0.071429,0.071429,良,...,障害,2850,良,-1.2,2-2-1-1,13.1,750.0,14.0,0.142857,0.071429
2005105204,201604010301,2016-05-07,4,雨,1,12,2.8,0.076923,0.307692,稍,...,障害,2890,稍,1.7,1-1-1-1,13.4,110.0,13.0,0.076923,0.076923
2005105204,201603010204,2016-04-10,3,晴,4,1,2.4,0.071429,0.285714,良,...,障害,2750,良,0.9,7-6-5-5,13.1,110.0,14.0,0.500000,0.357143
2005105204,201607020205,2016-03-13,7,曇,5,1,6.9,0.214286,0.142857,良,...,障害,3000,良,0.1,9-8-5-2,13.2,300.0,14.0,0.642857,0.142857


In [14]:
r.merge_horse_results(hr)
r.data_m.head()

  0%|          | 0/652 [00:00<?, ?it/s]

,horse_no,frame_no,arriving_order,horse_id,impost,jockey_id,trainer_id,date,place_id,hold_no,...,prise_9R,arriving_order_allR,popularity_allR,distance_allR,goal_time_allR,time_diff_allR,last_three_furlong_allR,first_corner_allR,last_corner_allR,prise_allR
race_id,,,,,,,,,,,,,,,,,,,,,
201601010101,6,6,1,2014102322,52.0,01151,01024,2016-07-30,1,1,...,0.0,0.598901,0.780220,1800.0,6.222222,1.25,35.75,0.964286,0.780220,0.0
201601010101,2,2,2,2014105971,54.0,00700,01066,2016-07-30,1,1,...,0.0,0.538462,0.230769,1600.0,6.037500,1.30,36.30,0.461538,0.615385,0.0
201601010101,5,5,3,2014100723,54.0,01091,00359,2016-07-30,1,1,...,65.0,0.416667,0.333333,1100.0,6.014167,1.05,35.90,0.430556,0.430556,65.0
201601010101,4,4,4,2014105192,51.0,01163,01046,2016-07-30,1,1,...,110.0,0.444444,0.666667,1800.0,6.272222,0.50,35.00,0.222222,0.555556,110.0
201601010101,3,3,5,2014106056,54.0,01014,01058,2016-07-30,1,1,...,280.0,0.133333,0.200000,1800.0,6.216667,0.50,35.40,0.600000,0.666667,280.0


In [6]:
r.data_m.to_pickle('./results_m_2015_2021.pickle')

In [8]:
p = Peds.read_db(db_config['main'])
p.data_e

,father,mother,fathers_father,fathers_mother,mothers_father,mothers_mother
id,,,,,,
2018105875,12463,7247,9681,10994,5087,11550
2018105258,6465,17257,10440,10500,13211,17000
2018101134,18431,9790,13211,11109,8323,15079
2018102791,2018,13761,1866,4508,1684,1096
2018101539,7479,15301,12463,12796,17437,15330
...,...,...,...,...,...,...
2019101670,7004,19023,11193,6964,14392,19027
2019103065,19198,20276,11033,18962,9339,4935
2019102321,21011,10151,7909,20688,9339,6900


In [15]:
r.merge_peds(p)
r.data_pe.head()

,horse_no,frame_no,arriving_order,horse_id,impost,jockey_id,trainer_id,date,place_id,hold_no,...,last_three_furlong_allR,first_corner_allR,last_corner_allR,prise_allR,father,mother,fathers_father,fathers_mother,mothers_father,mothers_mother
race_id,,,,,,,,,,,,,,,,,,,,,
201601010101,6,6,1,2014102322,52.0,01151,01024,2016-07-30,1,1,...,35.75,0.964286,0.780220,0.0,5015,15723,6698,17596,3690,4118
201601010101,2,2,2,2014105971,54.0,00700,01066,2016-07-30,1,1,...,36.30,0.461538,0.615385,0.0,14824,10704,923,2998,9681,15588
201601010101,5,5,3,2014100723,54.0,01091,00359,2016-07-30,1,1,...,35.90,0.430556,0.430556,65.0,4653,17643,1341,213,11116,11786
201601010101,4,4,4,2014105192,51.0,01163,01046,2016-07-30,1,1,...,35.00,0.222222,0.555556,110.0,6710,16937,4257,4181,18219,11138
201601010101,3,3,5,2014106056,54.0,01014,01058,2016-07-30,1,1,...,35.40,0.600000,0.666667,280.0,14787,14494,9681,4644,1573,1696


In [16]:
r.process_categorical()
r.data_c.head()

,horse_no,frame_no,arriving_order,horse_id,impost,jockey_id,trainer_id,date,place_id,hold_no,...,last_three_furlong_allR,first_corner_allR,last_corner_allR,prise_allR,father,mother,fathers_father,fathers_mother,mothers_father,mothers_mother
race_id,,,,,,,,,,,,,,,,,,,,,
201601010101,6,6,1,9481,52.0,112,300,2016-07-30,1,1,...,35.75,0.964286,0.780220,0.0,5015,15723,6698,17596,3690,4118
201601010101,2,2,2,12002,54.0,11,51,2016-07-30,1,1,...,36.30,0.461538,0.615385,0.0,14824,10704,923,2998,9681,15588
201601010101,5,5,3,8447,54.0,63,301,2016-07-30,1,1,...,35.90,0.430556,0.430556,65.0,4653,17643,1341,213,11116,11786
201601010101,4,4,4,11372,51.0,124,43,2016-07-30,1,1,...,35.00,0.222222,0.555556,110.0,6710,16937,4257,4181,18219,11138
201601010101,3,3,5,12079,54.0,26,302,2016-07-30,1,1,...,35.40,0.600000,0.666667,280.0,14787,14494,9681,4644,1573,1696


In [17]:
tmp_df = r.target_binary()
tmp_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 276689 entries, 201601010101 to 202106050912
Data columns (total 57 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   horse_no                 276689 non-null  int64         
 1   frame_no                 276689 non-null  int64         
 2   impost                   276689 non-null  float64       
 3   jockey_id                276689 non-null  int32         
 4   trainer_id               276689 non-null  int32         
 5   date                     276689 non-null  datetime64[ns]
 6   place_id                 276689 non-null  int32         
 7   hold_no                  276689 non-null  int64         
 8   hold_day                 276689 non-null  int64         
 9   race_no                  276689 non-null  int64         
 10  distance                 276689 non-null  int64         
 11  race_type                276689 non-null  int64         
 12  turn

In [18]:
train = tmp_df[tmp_df['date'] <= dt.datetime(2020, 12, 31)]
test = tmp_df[tmp_df['date'] >= dt.datetime(2021, 1, 1)]

In [19]:
train, valid = split_data(train, test_size=0.2)

In [20]:
X_train = train.drop(['date', 'rank'], axis=1)
y_train = train['rank']
X_valid = valid.drop(['date', 'rank'], axis=1)
y_valid = valid['rank']
X_test = test.drop(['date', 'rank'], axis=1)
y_test = test['rank']

In [24]:
lgb_o_train = lgb_o.Dataset(X_train, y_train)
lgb_o_valid = lgb_o.Dataset(X_valid, y_valid, reference=lgb_o_train)

In [ ]:
params = {
    'objective': 'binary',
    'random_state': 100,
}

cat_list = ['jockey_id', 'trainer_id', 'place_id', 'race_type', 'turn', 'ground', 'weather', 'sex', 'father', 'mother', 'fathers_father', 'fathers_mother', 'mothers_father', 'mothers_mother']

lgb_clf_o = lgb_o.train(params, lgb_o_train, valid_sets=[lgb_o_train, lgb_o_valid], verbose_eval=100, early_stopping_rounds=50, categorical_feature=cat_list)

In [27]:
lgb_clf_o.params

{'objective': 'binary',
 'random_state': 100,
 'feature_pre_filter': False,
 'lambda_l1': 9.853293111478425,
 'lambda_l2': 8.095924071958757,
 'num_leaves': 8,
 'feature_fraction': 0.4,
 'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'min_child_samples': 20,
 'num_iterations': 1000,
 'early_stopping_round': 50,
 'categorical_column': [3, 4, 5, 10, 11, 12, 13, 16, 49, 50, 51, 52, 53, 54]}

In [21]:
params = {
    'objective': 'binary',
    'random_state': 100,
    'feature_pre_filter': False,
    'lambda_l1': 9.853293111478425,
    'lambda_l2': 8.095924071958757,
    'num_leaves': 8,
    'feature_fraction': 0.4,
    'bagging_fraction': 1.0,
    'bagging_freq': 0,
    'min_child_samples': 20,
    'num_iterations': 1000,
    'early_stopping_round': 50,
    'categorical_column': [3, 4, 5, 10, 11, 12, 13, 16, 49, 50, 51, 52, 53, 54]
}

In [22]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_valid = lgb.Dataset(X_test, y_test, reference=lgb_train)

#model = lgb.train(lgb_clf_o.params, lgb_train, valid_sets=lgb_valid, verbose_eval=100)
model = lgb.train(params, lgb_train, valid_sets=lgb_valid, verbose_eval=100)

c:\work\KeibaAI\.venv\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
c:\work\KeibaAI\.venv\lib\site-packages\lightgbm\engine.py:240: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 39731, number of negative: 147113
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.024637 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26520
[LightGBM] [Info] Number of data points in the train set: 186844, number of used features: 55
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.212643 -> initscore=-1.309069
[LightGBM] [Info] Start training from score -1.309069
Training until validation scores don't improve for 50 rounds
[100]	valid_0's binary_logloss: 0.460226
Early stopping, best iteration is:
[130]	valid_0's binary_logloss: 0.459887


In [23]:
y_pred_train = model.predict(X_train, num_iteration=model.best_iteration)
y_pred = model.predict(X_test, num_iteration=model.best_iteration)

In [24]:
from sklearn.metrics import roc_curve, roc_auc_score

In [25]:
roc_auc_score(y_train, y_pred_train)

0.7978933336373544

In [26]:
roc_auc_score(y_test, y_pred)

0.7400834267826674

In [33]:
importances = pd.DataFrame(model.feature_importance(), index=X_train.columns, columns=['importance'])
importances.sort_values('importance', ascending=False)[:20]

,importance
mothers_mother,162
mother,161
mothers_father,65
jockey_id,65
trainer_id,63
win_prise,61
horse_num,35
father,33
fathers_mother,30
arriving_order_5R,23


In [5]:
rc = RaceCard.scrape(['202206010611'], 20220116)
rc.data_p

  0%|          | 0/1 [00:00<?, ?it/s]

,horse_no,frame_no,horse_id,impost,jockey_id,trainer_id,date,place_id,hold_no,hold_day,...,turn,ground,weather,horse_num,month,sex,age,weight,weight_change,win_prise
race_id,,,,,,,,,,,,,,,,,,,,,
202206010611,1,1,2019103096,56.0,00422,00423,2022-01-16,6,1,6,...,右,良,晴,16,20,牡,3,500,2,4000
202206010611,2,1,2019105976,54.0,01122,01176,2022-01-16,6,1,6,...,右,良,晴,16,20,牝,3,450,2,4000
202206010611,3,2,2019104569,56.0,01091,01052,2022-01-16,6,1,6,...,右,良,晴,16,20,牡,3,530,8,4000
202206010611,4,2,2019104767,56.0,01093,01062,2022-01-16,6,1,6,...,右,良,晴,16,20,牡,3,440,-4,4000
202206010611,5,3,2019105169,56.0,05339,01127,2022-01-16,6,1,6,...,右,良,晴,16,20,牡,3,498,16,4000
202206010611,6,3,2019105807,56.0,01188,01081,2022-01-16,6,1,6,...,右,良,晴,16,20,牡,3,488,4,4000
202206010611,7,4,2019101642,56.0,01154,01117,2022-01-16,6,1,6,...,右,良,晴,16,20,牡,3,512,-6,4000
202206010611,8,4,2019100126,56.0,01180,00429,2022-01-16,6,1,6,...,右,良,晴,16,20,牡,3,504,4,4000
202206010611,9,5,2019102490,56.0,01096,01138,2022-01-16,6,1,6,...,右,良,晴,16,20,牡,3,492,10,4000


In [27]:
rc.merge_horse_results(hr)
rc.merge_peds(p)

  0%|          | 0/1 [00:00<?, ?it/s]

In [28]:
rc.process_categorical(r)
rc.data_c

,horse_no,frame_no,horse_id,impost,jockey_id,trainer_id,date,place_id,hold_no,hold_day,...,last_three_furlong_allR,first_corner_allR,last_corner_allR,prise_allR,father,mother,fathers_father,fathers_mother,mothers_father,mothers_mother
race_id,,,,,,,,,,,,,,,,,,,,,
202206010611,1,1,32621,56.0,1,40,2022-01-16,6,1,6,...,35.166667,0.143519,0.104167,273.333333,13766,20509,7909,4967,12463,5861
202206010611,2,1,33957,54.0,86,224,2022-01-16,6,1,6,...,35.300000,0.272727,0.272727,700.000000,7672,11048,16509,10151,1704,5723
202206010611,3,2,33226,56.0,63,101,2022-01-16,6,1,6,...,34.900000,0.285714,0.357143,700.000000,13211,11714,9681,5931,1166,1721
202206010611,4,2,33323,56.0,65,110,2022-01-16,6,1,6,...,35.300000,0.306818,0.306818,553.500000,7672,10682,16509,10151,21295,14940
202206010611,5,3,33548,56.0,181,175,2022-01-16,6,1,6,...,33.350000,0.406250,0.375000,956.200000,14824,8258,923,2998,13211,8254
202206010611,6,3,33903,56.0,149,129,2022-01-16,6,1,6,...,36.100000,0.475568,0.438531,187.000000,6255,17032,7916,18680,10669,12114
202206010611,7,4,32087,56.0,115,165,2022-01-16,6,1,6,...,35.740000,0.416484,0.287912,208.000000,20372,12444,7909,6126,12588,20949
202206010611,8,4,31500,56.0,141,44,2022-01-16,6,1,6,...,35.350000,0.745192,0.528846,345.000000,6465,5165,10440,10500,11833,15239
202206010611,9,5,32412,56.0,67,186,2022-01-16,6,1,6,...,39.633333,0.286905,0.332143,310.000000,7672,15101,16509,10151,3044,1198


In [30]:
rc.get_final_data().info()

<class 'pandas.core.frame.DataFrame'>
Index: 16 entries, 202206010611 to 202206010611
Data columns (total 56 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   horse_no                 16 non-null     int32         
 1   frame_no                 16 non-null     int32         
 2   impost                   16 non-null     float64       
 3   jockey_id                16 non-null     int32         
 4   trainer_id               16 non-null     int32         
 5   date                     16 non-null     datetime64[ns]
 6   place_id                 16 non-null     int64         
 7   hold_no                  16 non-null     int64         
 8   hold_day                 16 non-null     int64         
 9   race_no                  16 non-null     int64         
 10  distance                 16 non-null     int64         
 11  race_type                16 non-null     int64         
 12  turn                  

In [33]:
rc_test = rc.data_c.drop(['horse_id', 'date'], axis=1)
rc_pred = model.predict(rc_test, num_iteration=model.best_iteration)
rc_pred

array([0.17966791, 0.14393558, 0.22339995, 0.20571326, 0.3189501 ,
       0.14335064, 0.21226724, 0.21825378, 0.1285716 , 0.16017655,
       0.31544757, 0.351233  , 0.098194  , 0.26100754, 0.35708403,
       0.10966265])

In [36]:
df = rc.data.copy()
df['pred'] = rc_pred
df[['枠', '馬番', '馬名', 'pred']].sort_values('pred', ascending=False)

,枠,馬番,馬名,pred
14,8,15,ロジハービン,0.357084
11,6,12,サンストックトン,0.351233
4,3,5,アライバル,0.318950
10,6,11,テンダンス,0.315448
13,7,14,ホウオウプレミア,0.261008
2,2,3,フジマサフリーダム,0.223400
7,4,8,ヴェローナシチー,0.218254
6,4,7,タイセイディバイン,0.212267
3,2,4,ヴェールランス,0.205713
0,1,1,ニシノムネヲウツ,0.179668
